In [1]:
import pandas as pd

# Import the JCAMP-DX files into a dataframe

In [2]:
import os
import jcamp
import pandas as pd

# Directory containing the JCAMP-DX files
dir = "nist_files"

# Initialize an empty list to store data from each file
all_data = []

# Loop through all .jdx files in the directory
for filename in os.listdir(dir):
    file_path = os.path.join(dir, filename)
    
    # Read the JCAMP-DX file
    jdx_data = jcamp.jcamp_readfile(file_path)
    
    # Extract all available data
    data = {'filename': filename}
    for key, value in jdx_data.items():
        data[key] = value
    
    all_data.append(data)

# Create a pandas DataFrame from the collected data
df = pd.DataFrame(all_data)

# save to csv
df


/var/folders/83/q6p745y53rl_50j5g1hmd6540000gn/T/ipykernel_5833/3285823054.py:1: DtypeWarning: Columns (70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nist_data_with_smiles.csv')


,filename,title,jcamp-dx,data type,origin,owner,cas registry no,molform,$nist source,state,...,interferogram zerofill,spectral interval after zerofilling,apodization,folding limits,number of interferograms averaged per single channel spectrum,$spectra version,$uncertainty in y,sample description,pressure,temperature
0,nist_files/file_8028.jdx,2-Ethoxycinnamic acid,4.24,INFRARED SPECTRUM,EPA-IR VAPOR PHASE LIBRARY,SRD/NIST\nCollection (C) 2018 copyright by the...,69038-81-9,C 11 H 12 O 3,MSDC-IR,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nist_files/file_12061.jdx,DIAMYL MALEATE,4.24,INFRARED SPECTRUM,DOW CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,10099-71-5,C14 H24 O4,COBLENTZ,"SOLUTION (10% CCl4 FOR 2.5-7.5, 10% CS2 FOR 7....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nist_files/file_14410.jdx,"M-NITRO CARBANILIC ACID, n-UNDECYL ESTER",4.24,INFRARED SPECTRUM,"CITRUS EXPERIMENT STATION, UNIVERSITY OF FLORI...",COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,95126-58-2,C18 H28 N2 O4,COBLENTZ,SOLID (1.7 mg / 200 mg KBr),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nist_files/file_6005.jdx,"ACETOPHENONE, 2,4,5-TRIHYDROXY",4.24,INFRARED SPECTRUM,"TENNESSEE EASTMAN COMPANY, RESEARCH LABORATORIES",COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,1818-27-5,C8 H8 O4,COBLENTZ,SOLID (KBr PELLET),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nist_files/file_9336.jdx,"GLYCINONITRILE, 2,2-DIPHENYL-",4.24,INFRARED SPECTRUM,WYANDOTTE CHEMICALS CORP.,Copyright (C) 1987 by the Coblentz Society\nCo...,52460-99-8,C14 H12 N2,COBLENTZ,SOLID (OIL MULL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15767,nist_files/file_13366.jdx,1-Benzyl-5-phenylbarbituric acid,4.24,INFRARED SPECTRUM,EPA-IR VAPOR PHASE LIBRARY,SRD/NIST\nCollection (C) 2018 copyright by the...,72846-00-5,C 17 H 14 N 2 O 3,MSDC-IR,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15768,nist_files/file_7302.jdx,"2,4,6-trichloroborazine",4.24,INFRARED SPECTRUM,NaN,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,933-18-6,H3 B3 Cl3 N3,COBLENTZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15769,nist_files/file_8031.jdx,ETHYLBENZOYL ACETATE,4.24,INFRARED SPECTRUM,DOW CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,94-02-0,C11 H12 O3,COBLENTZ,"SOLUTION (10% CCl4 FOR 5000-1330, 10% CS2 FOR ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15770,nist_files/file_12078.jdx,"ACETIC ACID, NITRILOTRI-, TRISODIUM SALT",4.24,INFRARED SPECTRUM,STANDARD OIL OF OHIO,COBLENTZ SOC.\nCollection (C) 2018 copyright b...,NaN,C6 H6 N Na3 O6,COBLENTZ,SOLID (OIL MULL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('nist_data.csv', index=False)

# Obtain SMILES

In [ ]:
df = pd.read_csv('nist_data.csv')

### 1) Retrieve InChI from NIST webpage, then convert to SMILES

In [3]:
import requests
from bs4 import BeautifulSoup
from rdkit import Chem
import time
import json
from tqdm import tqdm

def get_inchi_from_nist(cas_number):
    url = f"https://webbook.nist.gov/cgi/inchi?ID={cas_number}&Mask=80"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script_elements = soup.find_all('script', type='application/ld+json')
        for script_element in script_elements:
            try:
                json_data = json.loads(script_element.string)
                inchi = json_data.get('inChI')
                if inchi:
                    return inchi
            except json.JSONDecodeError:
                continue
    return None

def inchi_to_smiles(inchi):
    mol = Chem.MolFromInchi(inchi)
    if mol:
        return Chem.MolToSmiles(mol)
    return None

# Import the ids.csv file
ids_df = pd.read_csv('ids.csv')

for index, row in tqdm(df.iterrows(), total=len(df), desc="Updating SMILES"):
    # Get the file number from the filename
    file_number = int(row['filename'].split('_')[-1].split('.')[0])
        
    # Get the corresponding row from ids_df (file number minus 1)
    corresponding_row = ids_df.iloc[file_number - 1]
    
    # Get the CAS number from the 'NIST ID' column
    cas_number = corresponding_row['NIST ID']
    inchi = get_inchi_from_nist(cas_number)
    if inchi:
        smiles = inchi_to_smiles(inchi)
        if smiles:
            df.at[index, 'smiles'] = smiles
            # print(f"Updated SMILES for index {index}, SMILES: {smiles}")
        else:
            print(f"Failed to convert InChI to SMILES for index {index}, CAS: {cas_number}")
            pass
    else:
        print(f"Failed to retrieve InChI for index {index}, CAS: {cas_number}")
        pass


df.to_csv('nist_data_with_smiles_updated.csv', index=False)


Updating SMILES:   1%|          | 137/15772 [00:17<32:28,  8.03it/s]

Failed to retrieve InChI for index 135, CAS: C3811049


Updating SMILES:   1%|          | 184/15772 [00:22<31:58,  8.13it/s][16:14:55] Explicit valence for atom # 13 N, 5, is greater than permitted
[16:14:55] ERROR: Explicit valence for atom # 13 N, 5, is greater than permitted

Updating SMILES:   1%|          | 186/15772 [00:23<32:06,  8.09it/s]

Failed to convert InChI to SMILES for index 184, CAS: C369380


Updating SMILES:   1%|▏         | 217/15772 [00:27<31:58,  8.11it/s][16:14:59] Explicit valence for atom # 6 N, 5, is greater than permitted
[16:14:59] ERROR: Explicit valence for atom # 6 N, 5, is greater than permitted

Updating SMILES:   1%|▏         | 219/15772 [00:27<32:06,  8.07it/s]

...

Updating SMILES:  99%|█████████▉| 15612/15772 [36:15<00:20,  7.84it/s]

Failed to retrieve InChI for index 15610, CAS: C1305620


Updating SMILES:  99%|█████████▉| 15639/15772 [36:18<00:16,  7.83it/s]

Failed to retrieve InChI for index 15637, CAS: C10043524


Updating SMILES: 100%|██████████| 15772/15772 [36:35<00:00,  7.18it/s]


### 2) Search up CAS numbers on CAS webpage

In [8]:
import requests
from tqdm import tqdm

def cas_to_smiles(cas_number):
    # PubChem PUG REST API URL for converting CAS to SMILES
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{cas_number}/property/CanonicalSMILES/JSON"
    
    # Send a request to PubChem
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract SMILES from the response
        smiles = data['PropertyTable']['Properties'][0]['CanonicalSMILES']
        return smiles
    else:
        return None

# Update SMILES for rows where it's NaN using CAS numbers
for index, row in tqdm(df[df['smiles'].isna()].iterrows(), total=df['smiles'].isna().sum(), desc="Updating SMILES from CAS"):
    cas_number = row['cas registry no']
    
    # Check if the CAS number is not NaN
    if pd.notna(cas_number):
        smiles = cas_to_smiles(cas_number)
        if smiles:
            df.at[index, 'smiles'] = smiles
        else:
            print(f"Failed to retrieve SMILES for index {index}, CAS: {cas_number}")
    else:
        print(f"CAS number is NaN for index {index}")



# Print the number of remaining NaN values in the SMILES column
remaining_nan = df['smiles'].isna().sum()
print(f"Number of remaining NaN values in SMILES column: {remaining_nan}")



Updating SMILES from CAS:   5%|▌         | 4/79 [00:00<00:06, 11.20it/s]

Failed to retrieve SMILES for index 184, CAS: 369-38-0
CAS number is NaN for index 217


Updating SMILES from CAS:   8%|▊         | 6/79 [00:00<00:07,  9.86it/s]

Failed to retrieve SMILES for index 813, CAS: 116436-30-7


Updating SMILES from CAS:  11%|█▏        | 9/79 [00:01<00:08,  8.03it/s]

...

CAS number is NaN for index 13281
CAS number is NaN for index 13376
CAS number is NaN for index 13402


Updating SMILES from CAS:  94%|█████████▎| 74/79 [00:07<00:00, 12.46it/s]

Failed to retrieve SMILES for index 15045, CAS: 7803-65-8


Updating SMILES from CAS: 100%|██████████| 79/79 [00:08<00:00,  9.55it/s]

CAS number is NaN for index 15456
Failed to retrieve SMILES for index 15610, CAS: 1305-62-0
Failed to retrieve SMILES for index 15637, CAS: 10043-52-4
Number of remaining NaN values in SMILES column: 36


### 3) Manually add SMILES for the remaining rows

In [79]:
# Get the filenumber from the filename
filenumber = int(df['filename'][15637].split('_')[-1].split('.')[0])

# Read the ids.csv file
ids_df = pd.read_csv('ids.csv')

# Get the NIST ID from the row number minus 1
nist_id = ids_df.iloc[filenumber - 1]['NIST ID']

print(f"Filenumber: {filenumber}")
print(f"NIST ID: {nist_id}")


Filenumber: 1217
NIST ID: C10043524


In [81]:
df.loc[184, 'smiles'] = 'C[N+](C)(C)CCOC1=CC=CC(=C1)[N+](=O)[O-].[Br-]'
df.loc[217, 'smiles'] = 'CCCCNOS(=O)(=O)CC(=O)O'
df.loc[813, 'smiles'] = 'C[N+](C)(C)CCOC1=CC=CC=C1[N+](=O)[O-].[Br-]'
df.loc[1374, 'smiles'] = '[NH4+].OS(=O)(=O)[O-]'
df.loc[1888, 'smiles'] = None
df.loc[2470, 'smiles'] = None
df.loc[3321, 'smiles'] = None
df.loc[4804, 'smiles'] = None
df.loc[4974, 'smiles'] = 'C[N+](C)(C)C1=CC=CC=C1.C1=CC=C(C=C1)S(=O)(=O)[O-]'
df.loc[5108, 'smiles'] = None
df.loc[5111, 'smiles'] = 'CCC1=CC=C(C=C1)CC'
df.loc[6206, 'smiles'] = None
df.loc[6970, 'smiles'] = None
df.loc[7191, 'smiles'] = None
df.loc[7643, 'smiles'] = None
df.loc[7942, 'smiles'] = None
df.loc[8302, 'smiles'] = 'CCS'
df.loc[8369, 'smiles'] = 'CCC(CC=C)O'
df.loc[8401, 'smiles'] = 'C[N+]1=C2C(=NC=NC2=[N+](N=C1)C)Cl.Cl'
df.loc[8486, 'smiles'] = None
df.loc[8538, 'smiles'] = '[NH4+].[NH4+].[O-]S(=O)(=O)OOS(=O)(=O)[O-]'
df.loc[8621, 'smiles'] = 'CCCC(C)N'
df.loc[8668, 'smiles'] = 'CCOC(=O)C1=CC=CO1'
df.loc[9024, 'smiles'] = None
df.loc[9246, 'smiles'] = '[NH4+].[NH4+].OP(=O)([O-])[O-]'
df.loc[9325, 'smiles'] = None
df.loc[11592, 'smiles'] = '[NH4+].[NH4+].[O-][Se](=O)(=O)[O-]'
df.loc[12028, 'smiles'] = None
df.loc[12804, 'smiles'] = '[NH4+].[NH4+].[O-]S(=O)(=S)[O-]'
df.loc[13281, 'smiles'] = '[NH3+]O[N+](O)([O-])O[N+](=O)[O-].[NH3+]O[N+](O)([O-])O[N+](=O)[O-].[N+](=O)(O)[O-].[N+](=O)(O)[O-].[Ce]'
df.loc[13376, 'smiles'] = None
df.loc[13402, 'smiles'] = 'CC.CCCCCCCCCCCCCCCCCC(=O)OC'
df.loc[15045, 'smiles'] = None
df.loc[15456, 'smiles'] = None
df.loc[15610, 'smiles'] = '[OH-].[OH-].[Ca+2]'
df.loc[15637, 'smiles'] = '[Cl-].[Cl-].[Ca+2]'

### Remaining rows with NaN SMILES

In [82]:
df[df['smiles'].isna()]

,filename,title,jcamp-dx,data type,origin,owner,cas registry no,molform,$nist source,state,...,spectral interval after zerofilling,apodization,folding limits,number of interferograms averaged per single channel spectrum,$spectra version,$uncertainty in y,sample description,pressure,temperature,smiles
1888,nist_files/file_68.jdx,Ammonium hydrogen difluoride,4.24,INFRARED SPECTRUM,Pacific Northwest National Laboratory Under IA...,Public domain,1341-49-7,H 5 F 2 N,IARPA-IR-S,solid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2470,nist_files/file_818.jdx,TRIMETHYLAMINE-TRIBORANE (7),4.24,INFRARED SPECTRUM,KALLERY CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,NaN,C3 H16 B3 N,COBLENTZ,"SOLID (KBr PELLET); $$ RESEARCH PURITY, CONTAI...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3321,nist_files/file_1141.jdx,BROMODIBORANE,4.24,INFRARED SPECTRUM,KALLERY CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,NaN,Br B2 H5,COBLENTZ,VAPOR AT 26 mmHg PRESSURE; $$ RESEARCH PURITY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4804,nist_files/file_34.jdx,POLYVINYL ALCOHOL,4.24,INFRARED SPECTRUM,"WYANDOTTE CHEMICALS CORP., WYANDOTTE, MICHIGAN...",COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,9002-89-5,NaN,COBLENTZ,SOLID (MINERAL OIL MULL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5108,nist_files/file_274.jdx,Aluminum Hydroxide,4.24,INFRARED SPECTRUM,Pacific Northwest National Laboratory Under IA...,Public domain,21645-51-2,Al H 3 O 3,IARPA-IR-S,solid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6206,nist_files/file_13050.jdx,"2,4-DICHLOROBENZYL(2-HYDROXYETHYL)DIMETHYL AMM...",4.24,INFRARED SPECTRUM,DOW CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,84309-33-1,C11 H16 Cl3 N O,COBLENTZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6970,nist_files/file_3655.jdx,MONOBROMOPENTABORANE,4.24,INFRARED SPECTRUM,KALLERY CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,NaN,B5 H8 Br,COBLENTZ,SOLUTION (CS2 SOLUTION VS. SOLVENT); $$ RESEAR...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7191,nist_files/file_306.jdx,ETHYLENETETRABORANE,4.24,INFRARED SPECTRUM,KALLERY CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,NaN,C2 H12 B4,COBLENTZ,VAPOR AT 30 mmHg PRESSURE; $$ RESEARCH PURITY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7643,nist_files/file_5153.jdx,MONOCHLORODECABORANE,4.24,INFRARED SPECTRUM,KALLERY CHEMICAL COMPANY,COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,NaN,B10 H13 Cl,COBLENTZ,SOLID (KBr PELLET),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7942,nist_files/file_6735.jdx,COBALT BLUE,4.24,INFRARED SPECTRUM,"DESOTO CHEMICAL COATINGS, INC.",COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,1333-88-6,NaN,COBLENTZ,SOLID (0.5 mg / 300 mg CsI DISC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


### Save the dataframe

In [83]:
df.to_csv('nist_data_with_smiles.csv', index=False)


# Preprocess the IR spectra

In [ ]:
df = pd.read_csv('nist_data_with_smiles.csv')

# Clean dataframe

In [ ]:
# Calculate the percentage of NaN values in each column
nan_percentage = df.isna().mean() * 100

# Sort the percentage of NaN values in descending order
nan_percentage = nan_percentage.sort_values(ascending=False)

# Print the percentage of NaN values for each column
for index, value in nan_percentage.items():
    original_index = df.columns.get_loc(index) + 1
    print(f"{value:.2f}%: {index} (Original Index: {original_index})")

In [ ]:
import matplotlib.pyplot as plt

# Create a horizontal bar plot for the percentage of NaN values in each column
plt.figure(figsize=(12, 20))
nan_percentage.plot(kind='barh')
plt.title('Percentage of NaN Values in Each Column')
plt.xlabel('Percentage of NaN Values')
plt.ylabel('Columns')
plt.show()

In [ ]:
# Get the columns with less than 10% NaN percentage
columns_less_than_10_percent_nan = nan_percentage[nan_percentage < 10].index.tolist()

# Filter the DataFrame to keep only these columns
df_filtered = df[columns_less_than_10_percent_nan]

# Display the filtered DataFrame
df_filtered
